## Use Google Earth Engine to export Sentinel2 data for classification

In [ ]:
import ee
import tensorflow as tf
from pprint import pprint
import folium
import time
import json

# Initialize Earth Engine and prep tiles for visualization
ee.Initialize()
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

tf.enable_eager_execution()

In [ ]:
# CHANGE TO YOUR GOOGLE CLOUD STORAGE BUCKET
OUTPUT_BUCKET = 'your_bucket'
BANDS = ['TCI_R', 'TCI_G', 'TCI_B'] # We want the true color bands
IMAGE_FILE_PREFIX = 'sentinel_2_rgb'
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00]) # Area to get data

In [ ]:
sent2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(EXPORT_REGION)

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000).select(BANDS)

In [ ]:
# The image input data is a 2018 cloud-masked median composite.
image = sent2.filterDate('2018-01-01', '2019-01-31').map(maskS2clouds).median()

In [ ]:
# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': BANDS})
map = folium.Map(location=[38., -122.5])
folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
### RUN EXPORT JOB TO GOOGLE CLOUD STORAGE BUCKET
## EXPORT AS A TF RECORD

# Specify patch and file dimensions.
imageExportFormatOptions = {
  'patchDimensions': [64, 64], # our model takes in 64x64 images
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the export task.
imageTask = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=10,
  fileFormat='TFRecord',
  region=EXPORT_REGION.toGeoJSON()['coordinates'],
  formatOptions=imageExportFormatOptions)
    
imageTask.start()

In [ ]:
# MONITOR EXPORT JOB
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(30)
print('Done with image export.')